In [4]:
import cv2
import numpy as np 
import torch 
from torchvision import transforms, models 
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import LatentDirichletAllocation 
from scipy.spatial import distance
from torchvision import transforms

In [28]:
from sklearn.tree import DecisionTreeClassifier

In [7]:
from torch.utils.data import Subset

In [9]:
from tqdm import tqdm

In [21]:
import utils

In [6]:
dataset = torchvision.datasets.Caltech101('/Users/niteeqsheik/Desktop/Phase3',download=True)
data_loader = torch.utils.data.DataLoader(dataset,
batch_size=4,
shuffle=True,
num_workers=8)
print(len(dataset))

131740031it [00:04, 26916279.73it/s]


Extracting /Users/niteeqsheik/Desktop/Phase3/caltech101/101_ObjectCategories.tar.gz to /Users/niteeqsheik/Desktop/Phase3/caltech101


14028800it [00:00, 18831818.28it/s]


Extracting /Users/niteeqsheik/Desktop/Phase3/caltech101/Annotations.tar to /Users/niteeqsheik/Desktop/Phase3/caltech101
8677


In [8]:
even_indices = [i for i in range(0, len(dataset), 2)]
even_dataset = Subset(dataset, even_indices)
even_data_loader = torch.utils.data.DataLoader(even_dataset, batch_size=4, shuffle=True, num_workers=8)
print(len(even_dataset))

4339


In [11]:
resnet50 = models.resnet50(pretrained = 50)

In [12]:
resnet50.eval() 

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [16]:
transform = transforms.Compose([
    transforms.Resize(256),             # Resize the image to 256x256 pixels
    transforms.CenterCrop(224),         # Crop the image to 224x224 pixels about the center
    transforms.Grayscale(num_output_channels=3),  
    transforms.ToTensor(),              # Convert the image to a tensor
    transforms.Normalize(mean=[0.53994344, 0.52009986, 0.49254049], std=[0.31415099, 0.30712622, 0.31878401]),  # Normalize the image
])
def extract_features_individual(even_dataset, model, transform):
    features = []
    labels = []
    with torch.no_grad():
        for image, label in tqdm(even_dataset, desc="Extracting Features"):
            image = transform(image).unsqueeze(0)  # Apply transform and add batch dimension
            output = model(image)
            features.append(output.squeeze(0)) 
            labels.append(label) # Remove batch dimension
    return features,labels
# Extract Features from Even-Numbered Images
even_features, even_labels = extract_features_individual(even_dataset, resnet50, transform)

Extracting Features: 100%|██████████| 4339/4339 [02:53<00:00, 25.08it/s]


In [20]:
# import matplotlib.pyplot as plt
# from torchvision import datasets, transforms
# from torch.utils.data import Subset
# import numpy as np

# # Assuming you have already defined and loaded your dataset and even_dataset

# # Function to show an image
# def imshow(img):
#     img = img / 2 + 0.5  # unnormalize
#     npimg = img.numpy()
#     plt.imshow(np.transpose(npimg, (1, 2, 0)))
#     plt.show()

# # Display three images for each label
# unique_labels = set(even_labels)

# for label in unique_labels:
#     print(f"Label: {label}")
#     count = 0
#     for i, (image, lbl) in enumerate(even_dataset):
#         if lbl == label and count < 3:
#             imshow(transforms.ToTensor()(image))
#             count += 1
#         if count >= 3:
#             break

### KNN Classifier for even numbered images

In [26]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np 
def prepare_knn_classifier(k,features,labels):
    """
    Prepares a k-NN classifier.

    Args:
    k (int): The number of neighbors to use for k-NN.
    features (list of torch.Tensor): The list of feature tensors from the ResNet model.
    labels (list): The list of labels corresponding to the features.

    Returns:
    KNeighborsClassifier: The trained k-NN classifier
    """
    knn = KNeighborsClassifier(n_neighbors = k) 
    knn.fit(features,labels)
    return knn 
db_path = 'caltech101_features.db'
even_features_db,even_labels_db = utils.load_even_images_and_labels(db_path)

#convert the list of numpy arrays to 2D numpy arrays 
X = np.stack(even_features_db)
y = np.array(even_labels_db)
k = int(input("Enter the number of neighbors for k-NN: "))
knn_classifier = prepare_knn_classifier(k,X,y)


### Decision Tree Classifier

In [29]:
def train_decision_tree(features, labels, **dt_params): 
    """
    Trains a Decision Tree classifier using all provided features and labels.

    Args:
    features (numpy.ndarray): The feature vectors.
    labels (numpy.ndarray): The labels corresponding to the features.
    **dt_params: Additional parameters to pass to the Decision Tree classifier.

    Returns:
    DecisionTreeClassifier: The trained Decision Tree classifier.
    """
    # Initialize the Decision Tree classifier
    decision_tree = DecisionTreeClassifier(**dt_params)

    # Train the classifier
    decision_tree.fit(features, labels)

    return decision_tree
decision_tree_classifier = train_decision_tree(X, y)

### PPR CODE

In [33]:
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity

In [43]:
import numpy as np 
features_array = np.stack(even_labels_db)
print(features_array.shape)

(4339,)


In [51]:
import numpy as np

# Check if even_features is a list of NumPy arrays
if len(even_features_db) > 0 and isinstance(even_features_db[0], np.ndarray):
    print("The elements of even_features are NumPy arrays.")
else:
    print("The elements of even_features are not NumPy arrays.")


The elements of even_features are NumPy arrays.


In [55]:
# import networkx as nx
# import numpy as np
# from sklearn.metrics.pairwise import cosine_similarity

# def create_similarity_graph(features):
#     """
#     Creates an undirected graph based on cosine similarity between features.

#     Args:
#     features (numpy.ndarray): Feature vectors of the images.

#     Returns:
#     networkx.Graph: An undirected graph representing the similarities between images.
#     """
#     similarity_matrix = cosine_similarity(features)
#     G = nx.Graph()
#     for i in range(similarity_matrix.shape[0]):
#         for j in range(similarity_matrix.shape[1]):
#             if i != j:  # Avoid self-loops
#                 G.add_edge(i, j, weight=similarity_matrix[i][j])
#     return G

# def personalized_pagerank_classifier(graph, labels, alpha=0.85, max_iter=500):
#     unique_labels = set(labels)
#     predictions = {}

#     for node in graph.nodes():
#         pagerank_scores = {}
#         for label in unique_labels:
#             personalization = {i: 0.1 if labels[i] != label else 1 for i in graph.nodes()}  # Softer personalization
#             try:
#                 pagerank = nx.pagerank(graph, alpha=alpha, personalization=personalization, max_iter=max_iter)
#                 pagerank_scores[label] = pagerank[node]
#                 print(f"Success: PageRank converged for label '{label}' at node {node}.")
#             except nx.PowerIterationFailedConvergence as e:
#                 print(f"Error: PageRank failed to converge for label '{label}' at node {node}. {e}")
#                 pagerank_scores[label] = 0  # Handle non-convergence

#         predicted_label = max(pagerank_scores, key=pagerank_scores.get)
#         predictions[node] = predicted_label

#     return predictions

# # Load your features and labels
# # even_features, even_labels = ...

# # Create a graph based on the features
# G = create_similarity_graph(np.stack(even_features_db))

# # Classify the images
# predictions = personalized_pagerank_classifier(G, even_labels_db)

# # Now, `predictions` contains the predicted labels for each image


In [6]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score
import utils
from sklearn.preprocessing import StandardScaler
from collections import Counter
from sklearn.metrics import accuracy_score, f1_score, recall_score
from sklearn.metrics import classification_report


class CustomKNN:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)

    def _predict(self, x):
        # Compute distances between x and all examples in the training set
        distances = [np.sqrt(np.sum((x_train - x) ** 2)) for x_train in self.X_train]
        # Sort by distance and return indices of the first k neighbors
        k_indices = np.argsort(distances)[:self.k]
        # Extract the labels of the k nearest neighbor training samples
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        # Return the most common class label
        most_common = Counter(k_nearest_labels).most_common(1)
        return most_common[0][0]


class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None


class CustomDecisionTree:
    def __init__(self, max_depth=10):
        self.max_depth = max_depth
        self.root = None

    def fit(self, X, y):
        if y.dtype.kind in 'UO':
            unique_labels, y = np.unique(y, return_inverse=True)
            self.label_dict = {i: label for i, label in enumerate(unique_labels)}
        else:
            self.label_dict = None

        self.n_classes_ = len(set(y))
        self.n_features_ = X.shape[1]
        self.feature_importances_ = np.zeros(self.n_features_)
        self.root = self._grow_tree(X, y)

    def predict(self, X):
        return np.array([self._predict(x, self.root) for x in X])
    

    def _grow_tree(self, X, y, depth=0):
        num_samples_per_class = [np.sum(y == c) for c in np.unique(y)]
        predicted_class = np.argmax(num_samples_per_class)

        # stopping criteria
        if depth >= self.max_depth or len(np.unique(y)) == 1:
            return Node(value=predicted_class)

        feature_idxs = np.arange(X.shape[1])
        best_feature, best_thresh = self._best_criteria(X, y, feature_idxs)

        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left_child = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right_child = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        return Node(best_feature, best_thresh, left_child, right_child)

    def _best_criteria(self, X, y, feature_idxs):
        best_gain = -1
        split_idx, split_thresh = None, None
        for feature_idx in feature_idxs:
            X_column = X[:, feature_idx]
            thresholds = np.unique(X_column)
            for threshold in thresholds:
                gain = self._information_gain(y, X_column, threshold)
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feature_idx
                    split_thresh = threshold
        return split_idx, split_thresh

    def _information_gain(self, y, X_column, split_thresh):
        # parent loss
        parent_entropy = entropy(y)

        # generate split
        left_idxs, right_idxs = self._split(X_column, split_thresh)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # compute the weighted avg. of the loss for the children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = entropy(y[left_idxs]), entropy(y[right_idxs])
        child_entropy = (n_l / n) * e_l + (n_r / n) * e_r

        # information gain is difference in loss before vs. after split
        ig = parent_entropy - child_entropy
        return ig

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _predict(self, x, node):
        if node.is_leaf_node():
            return node.value
        if x[node.feature] <= node.threshold:
            return self._predict(x, node.left)
        return self._predict(x, node.right)
class ImageClassifier: 

    def __init__(self, **dt_params): 
        self.dt_params = dt_params 
        self.classifier = None
        self.scaler = StandardScaler() 

    def train(self,features,labels):
        features = self.scaler.fit_transform(features) 
        self.classifier = DecisionTreeClassifier(**self.dt_params) 
        self.classifier.fit(features,labels) 

    def predict_and_evaluate(self,features,true_labels):
        features = self.scaler.transform(features)
        predicted_labels = self.classifier.predict(features)
        # Print predictions for each odd image

        for image_id, label in enumerate(predicted_labels, start=1):
            if image_id % 2 != 0:  # Check if the image ID is odd
                print(f"Odd Image ID {image_id}: Predicted Label - {label}")
        # Evaluate performance

        accuracy = accuracy_score(true_labels, predicted_labels)
        class_report = classification_report(true_labels, predicted_labels)
        # Print overall accuracy
        print("Overall Accuracy:", accuracy)
        # Print per-label precision, recall, and F1-score
        print("\nClassification Report:")
        print(class_report)

        # f1 = f1_score(true_labels, predicted_labels, average='weighted')
        # recall = recall_score(true_labels, predicted_labels, average='weighted')
        # return {"accuracy": accuracy, "f1_score": f1, "recall": recall}



def entropy(y):
    hist = np.bincount(y)
    ps = hist / len(y)
    return -np.sum([p * np.log2(p) for p in ps if p > 0])

even_features, even_labels = utils.load_even_images_and_labels('caltech101_features.db')

# Standardize features
scaler = StandardScaler()
X_even = scaler.fit_transform(even_features)
y_even = np.array(even_labels)

# User input for classifier selection
classifier_choice = input("1) k-NN\n2) Decision Tree\n3) PPR\n\nSelect classifier: ").strip()
flag = 0 
# Classifier initialization
if classifier_choice == '1':
    k = int(input("Enter the number of neighbors for k-NN: "))
    classifier = CustomKNN(k)
    classifier.fit(X_even, y_even)
    flag = 1 
elif classifier_choice == '2':
    classifier = ImageClassifier(max_depth = 10)
    classifier.train(even_features,even_labels)
    flag = 2 
else:
    print("Invalid classifier choice.")
    exit()

# Load odd features and labels
odd_features, odd_labels = utils.load_odd_images_and_labels('caltech101_features.db')

# Standardize odd features
X_odd = scaler.transform(odd_features)
y_odd = np.array(odd_labels)
if flag ==2 : 
    odd_labels = np.array(odd_labels) 
    # evaluation_results = classifier.predict_and_evaluate(odd_features, odd_labels)
    # print(evaluation_results)
    classifier.predict_and_evaluate(odd_features, odd_labels)
# Predict labels for odd-numbered images
if flag ==1 :
    predicted_labels = classifier.predict(X_odd)

    # Print predictions for each odd image
    odd_image_ids = np.arange(1, len(odd_features) * 2, 2)  # Assuming odd indices represent odd-numbered images
    for image_id, label in zip(odd_image_ids, predicted_labels):
        print(f"Odd Image ID {image_id}: Label - {label}")

    # Classification report and accuracy
    print("\nClassification Report for Odd-Numbered Images:")
    print(classification_report(y_odd, predicted_labels, labels=np.unique(y_odd)))
    print("Overall Accuracy:", accuracy_score(y_odd, predicted_labels))


Odd Image ID 1: Predicted Label - 0
Odd Image ID 3: Predicted Label - 0
Odd Image ID 5: Predicted Label - 1
Odd Image ID 7: Predicted Label - 0
Odd Image ID 9: Predicted Label - 0
Odd Image ID 11: Predicted Label - 0
Odd Image ID 13: Predicted Label - 0
Odd Image ID 15: Predicted Label - 0
Odd Image ID 17: Predicted Label - 0
Odd Image ID 19: Predicted Label - 0
Odd Image ID 21: Predicted Label - 1
Odd Image ID 23: Predicted Label - 1
Odd Image ID 25: Predicted Label - 0
Odd Image ID 27: Predicted Label - 0
Odd Image ID 29: Predicted Label - 1
Odd Image ID 31: Predicted Label - 23
Odd Image ID 33: Predicted Label - 1
Odd Image ID 35: Predicted Label - 0
Odd Image ID 37: Predicted Label - 0
Odd Image ID 39: Predicted Label - 0
Odd Image ID 41: Predicted Label - 0
Odd Image ID 43: Predicted Label - 1
Odd Image ID 45: Predicted Label - 11
Odd Image ID 47: Predicted Label - 0
Odd Image ID 49: Predicted Label - 0
Odd Image ID 51: Predicted Label - 1
Odd Image ID 53: Predicted Label - 1
Odd 

/Users/niteeqsheik/anaconda3/envs/WebDB_Project/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/niteeqsheik/anaconda3/envs/WebDB_Project/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/niteeqsheik/anaconda3/envs/WebDB_Project/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_